In [1]:
# Import libraries
from pathlib import Path
import sys
import logging
import warnings
import json

import numpy as np
import pandas as pd

# Suppress warnings for cleaner output
warnings.filterwarnings("ignore")
logging.basicConfig(level=logging.INFO, format="%(levelname)s: %(message)s")

# Helper: find project root by looking for specified directories
def find_project_root(start: Path) -> Path:
    """Find the project root directory (contains 'config' and 'src' folders)."""
    start = start.resolve()
    for p in [start, *start.parents]:
        if (p / "config").exists() and (p / "src").exists():
            return p
    raise RuntimeError("Project root not found (expected 'config' and 'src' directories).")

# Helper: persist a dictionary as a JSON file (with indentation for readability)
def _persist_json(obj: dict, path: Path) -> None:
    path.parent.mkdir(parents=True, exist_ok=True)
    # Convert any numpy types to native types for JSON serialization
    obj_converted = {k: (float(v) if isinstance(v, np.generic) else v) for k, v in obj.items()}
    path.write_text(json.dumps(obj_converted, indent=2), encoding="utf-8")


In [2]:
# Define paths for data and metrics
ROOT = find_project_root(Path.cwd())
ARTIFACTS_DIR = ROOT / "artifacts"
FEATURES_PATH = ARTIFACTS_DIR / "data" / "features_monthly.parquet"
METRICS_DIR = ARTIFACTS_DIR / "metrics"
METRICS_DIR.mkdir(parents=True, exist_ok=True)

# Load dataset
if not FEATURES_PATH.exists():
    raise FileNotFoundError(f"Features file not found at {FEATURES_PATH}. Please run notebooks 00-03 to generate it.")
df = pd.read_parquet(FEATURES_PATH)

# Ensure the index is a DateTime index and sorted
if not isinstance(df.index, pd.DatetimeIndex):
    raise TypeError("Index must be DatetimeIndex for time-series modeling.")
if not df.index.is_monotonic_increasing:
    df = df.sort_index()

# Verify required target columns are present
required_targets = {"y_return_next_pct", "y_direction_next"}
missing_targets = required_targets - set(df.columns)
if missing_targets:
    raise KeyError(f"Missing target columns in features data: {missing_targets}")

# Display the date range of the data for reference
print(f"Data date range: {df.index.min().date()} → {df.index.max().date()} (n={len(df)})")

# Train-test split (using the conventional split: train up to 2019-12, test from 2020-01)
train = df[df.index <= "2019-12-31"]
test  = df[df.index >= "2020-01-31"]
print(f"Train period: {train.index.min().date()} → {train.index.max().date()} | n={len(train)}")
print(f"Test  period: {test.index.min().date()} → {test.index.max().date()} | n={len(test)}")

# Separate features (X) and targets (y) for both regression and classification
X_train = train.drop(columns=list(required_targets))
X_test  = test.drop(columns=list(required_targets))
y_train_reg = train["y_return_next_pct"]
y_test_reg  = test["y_return_next_pct"]
# Ensure classification target is integer type
y_train_clf = train["y_direction_next"].astype(int)
y_test_clf  = test["y_direction_next"].astype(int)


Data date range: 2009-02-28 → 2025-05-31 (n=196)
Train period: 2009-02-28 → 2019-12-31 | n=131
Test  period: 2020-01-31 → 2025-05-31 | n=65


In [7]:
import ipywidgets as widgets
import matplotlib.pyplot as plt
from IPython.display import display, clear_output
from sklearn.linear_model import LinearRegression, Ridge, LogisticRegression
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
from sklearn.metrics import accuracy_score, f1_score, precision_score, recall_score, roc_auc_score, roc_curve

# Define the evaluation function that trains models and outputs metrics and plots
def evaluate_model(ridge_alpha=1.0, logistic_C=1.0, threshold=0.5):
    # Train models with given parameters
    linreg = LinearRegression()
    ridge = Ridge(alpha=ridge_alpha)
    logreg = LogisticRegression(C=logistic_C, solver='lbfgs', max_iter=1000, random_state=0)
    linreg.fit(X_train, y_train_reg)
    ridge.fit(X_train, y_train_reg)
    logreg.fit(X_train, y_train_clf)
    # Generate predictions on the test set
    y_pred_ols = linreg.predict(X_test)
    y_pred_ridge = ridge.predict(X_test)
    y_prob = logreg.predict_proba(X_test)[:, 1]  # probability of class 1 (Up) for each test sample
    y_pred_class = (y_prob >= threshold).astype(int)
    # Compute regression metrics
    rmse_ols = np.sqrt(mean_squared_error(y_test_reg, y_pred_ols))
    rmse_ridge = np.sqrt(mean_squared_error(y_test_reg, y_pred_ridge))
    mae_ols = mean_absolute_error(y_test_reg, y_pred_ols)
    mae_ridge = mean_absolute_error(y_test_reg, y_pred_ridge)
    r2_ols = r2_score(y_test_reg, y_pred_ols)
    r2_ridge = r2_score(y_test_reg, y_pred_ridge)
    # Compute classification metrics
    acc_log = accuracy_score(y_test_clf, y_pred_class)
    f1_log = f1_score(y_test_clf, y_pred_class)
    prec_log = precision_score(y_test_clf, y_pred_class, zero_division=0)
    rec_log = recall_score(y_test_clf, y_pred_class, zero_division=0)
    try:
        auc_log = roc_auc_score(y_test_clf, y_prob)
    except ValueError:
        # If only one class present in y_test (unlikely in our case), ROC AUC is undefined
        auc_log = float('nan')
    # Print out the metrics
    print("Regression performance on test set:")
    print(f"  RMSE (OLS) = {rmse_ols:.4f},   RMSE (Ridge, α={ridge_alpha}) = {rmse_ridge:.4f}")
    print(f"  MAE  (OLS) = {mae_ols:.4f},   MAE  (Ridge, α={ridge_alpha}) = {mae_ridge:.4f}")
    print(f"  R^2  (OLS) = {r2_ols:.4f},   R^2  (Ridge, α={ridge_alpha}) = {r2_ridge:.4f}")
    print("\nClassification performance on test set:")
    print(f"  Accuracy = {acc_log:.4f},   F1-score = {f1_log:.4f}")
    print(f"  Precision = {prec_log:.4f},   Recall = {rec_log:.4f},   ROC AUC = {auc_log:.4f}")
    # Plot Actual vs Predicted returns and ROC curve
    fig, axes = plt.subplots(1, 2, figsize=(14, 5))
    # Left plot: time series of actual vs predicted returns
    axes[0].plot(y_test_reg.index, y_test_reg, label="Actual", color='C0')
    axes[0].plot(y_test_reg.index, y_pred_ols, label="Predicted OLS", color='C1')
    axes[0].plot(y_test_reg.index, y_pred_ridge, label=f"Predicted Ridge (α={ridge_alpha})", color='C2')
    axes[0].axhline(0.0, color='k', linestyle='--', linewidth=0.8)  # reference line at 0% return
    axes[0].set_title("SP500 Monthly Returns: Actual vs Predicted")
    axes[0].set_xlabel("Date")
    axes[0].set_ylabel("Monthly Return (%)")
    axes[0].legend(loc="best")
    for label in axes[0].get_xticklabels():
        label.set_rotation(45)
    # Right plot: ROC curve for classification (if applicable)
    axes[1].set_title("ROC Curve (Logistic Regression)")
    axes[1].set_xlabel("False Positive Rate")
    axes[1].set_ylabel("True Positive Rate")
    # Plot ROC curve only if both classes are present in test set
    if len(np.unique(y_test_clf)) == 2:
        fpr, tpr, _ = roc_curve(y_test_clf, y_prob)
        axes[1].plot(fpr, tpr, label=f"Model (AUC = {auc_log:.2f})", color='C1')
        axes[1].plot([0, 1], [0, 1], 'k--', label="Random Chance")
        axes[1].legend(loc="lower right")
    else:
        axes[1].text(0.5, 0.5, "Only one class present in test data", ha='center', va='center')
    plt.tight_layout()
    plt.show()

# Create interactive widgets for parameters
alpha_slider = widgets.FloatLogSlider(value=1.0, base=10, min=-3, max=2, step=0.1, description="Ridge α")
c_slider = widgets.FloatLogSlider(value=1.0, base=10, min=-3, max=3, step=0.1, description="Logistic C")
threshold_slider = widgets.FloatSlider(value=0.5, min=0.0, max=1.0, step=0.01, description="Threshold")

# Display the interactive controls and output
widgets.interact(evaluate_model, ridge_alpha=alpha_slider, logistic_C=c_slider, threshold=threshold_slider);


interactive(children=(FloatLogSlider(value=1.0, description='Ridge α', max=2.0, min=-3.0), FloatLogSlider(valu…

In [4]:
# Set final chosen hyperparameters (modify these based on your exploration if desired)
final_ridge_alpha = 1.0
final_logistic_C = 1.0
final_threshold = 0.5

# Train final models with chosen parameters on the full training set
linreg_final = LinearRegression()
ridge_final = Ridge(alpha=final_ridge_alpha)
logreg_final = LogisticRegression(C=final_logistic_C, solver='lbfgs', max_iter=1000, random_state=0)
linreg_final.fit(X_train, y_train_reg)
ridge_final.fit(X_train, y_train_reg)
logreg_final.fit(X_train, y_train_clf)

# Make predictions on test set
y_pred_ols_final = linreg_final.predict(X_test)
y_pred_ridge_final = ridge_final.predict(X_test)
y_prob_final = logreg_final.predict_proba(X_test)[:, 1]
y_pred_class_final = (y_prob_final >= final_threshold).astype(int)

# Compute metrics for final model
metrics = {
    # Regression metrics (OLS vs Ridge)
    "REG_RMSE_ols": np.sqrt(mean_squared_error(y_test_reg, y_pred_ols_final)),
    "REG_RMSE_ridge": np.sqrt(mean_squared_error(y_test_reg, y_pred_ridge_final)),
    "REG_MAE_ols": mean_absolute_error(y_test_reg, y_pred_ols_final),
    "REG_MAE_ridge": mean_absolute_error(y_test_reg, y_pred_ridge_final),
    "REG_R2_ols": r2_score(y_test_reg, y_pred_ols_final),
    "REG_R2_ridge": r2_score(y_test_reg, y_pred_ridge_final),
    # Classification metrics (Logistic Regression)
    "CLF_Acc_logistic": accuracy_score(y_test_clf, y_pred_class_final),
    "CLF_F1_logistic": f1_score(y_test_clf, y_pred_class_final, zero_division=0),
    "CLF_Prec_logistic": precision_score(y_test_clf, y_pred_class_final, zero_division=0),
    "CLF_Recall_logistic": recall_score(y_test_clf, y_pred_class_final, zero_division=0),
    "CLF_AUC_logistic": roc_auc_score(y_test_clf, y_prob_final) if len(np.unique(y_test_clf)) == 2 else float('nan')
}

# Print the metrics for verification
for k, v in metrics.items():
    if isinstance(v, float) or isinstance(v, np.floating):
        print(f"{k}: {v:.4f}")
    else:
        print(f"{k}: {v}")

# Save metrics to a JSON file in the artifacts/metrics directory
out_path = METRICS_DIR / "linear.json"
_persist_json(metrics, out_path)
print(f"Metrics saved to: {out_path}")


REG_RMSE_ols: 5.1870
REG_RMSE_ridge: 5.1701
REG_MAE_ols: 4.3686
REG_MAE_ridge: 4.3741
REG_R2_ols: -0.0164
REG_R2_ridge: -0.0098
CLF_Acc_logistic: 0.4462
CLF_F1_logistic: 0.5000
CLF_Prec_logistic: 0.5806
CLF_Recall_logistic: 0.4390
CLF_AUC_logistic: 0.4654
Metrics saved to: C:\Users\gamer\Desktop\AktienPrognose\artifacts\metrics\linear.json
